In [168]:
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import plotly.express as px
import numpy as np
import folium

## Task 1,3

In [2]:
# import some datasets
electro_df = pd.read_csv('global_electricity.csv')

In [23]:
electro_df['1980'].value_counts()

1980
0          421
0          224
-1          28
0.002       11
0.005       11
          ... 
20.2         1
10.157       1
1.10577      1
16.765       1
5.145        1
Name: count, Length: 777, dtype: int64

In [68]:
# Get rid of strange behaviour
electro_df.fillna(0, inplace=True)
electro_df.replace({'--': 0, 'ie': 0, ',': '.'}, inplace=True)
for i in range(1980, 2022):
    electro_df[str(i)] = electro_df[str(i)].astype(np.float64)
electro_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1610 entries, 0 to 1609
Data columns (total 45 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Country   1610 non-null   object 
 1   Features  1610 non-null   object 
 2   Region    1610 non-null   object 
 3   1980      1610 non-null   float64
 4   1981      1610 non-null   float64
 5   1982      1610 non-null   float64
 6   1983      1610 non-null   float64
 7   1984      1610 non-null   float64
 8   1985      1610 non-null   float64
 9   1986      1610 non-null   float64
 10  1987      1610 non-null   float64
 11  1988      1610 non-null   float64
 12  1989      1610 non-null   float64
 13  1990      1610 non-null   float64
 14  1991      1610 non-null   float64
 15  1992      1610 non-null   float64
 16  1993      1610 non-null   float64
 17  1994      1610 non-null   float64
 18  1995      1610 non-null   float64
 19  1996      1610 non-null   float64
 20  1997      1610 non-null   floa

In [69]:
# box chart for electricity df
electro_df.fillna(-1, inplace=True)
box_fig = px.box(electro_df[(electro_df['Features'] == 'net generation') & (~electro_df['Country'].str.contains('Former'))], x='Region', y='2015')
box_fig.show()

In [67]:
# Histogram chart for electricity
region_electricity = electro_df[electro_df['Features'] == 'net generation'].groupby('Region').sum().reset_index()
region_electricity = region_electricity.loc[:, electro_df.columns.difference(['Country', 'Features'])]

px.histogram(region_electricity, x='Region', y='2020')

In [109]:
# Pie chart for energy producers
country_electricity = electro_df.query("Features == 'net consumption'")[['Country', '2020']]
country_electricity.loc[country_electricity['2020'] < 500, 'Country'] = 'Other'
pie_fig = px.pie(country_electricity, labels='Country', values='2020', title='Energy consumers of 2020', names='Country')
pie_fig.update_traces(textinfo='percent+label')

In [144]:
scatter_df = electro_df.query("Features == 'net generation'").groupby('Region').sum()
scatter_df = scatter_df[electro_df.columns.difference(['Country', 'Features', 'Region'])].transpose().reset_index()
scatter_fig = go.Figure()
scatter_fig.add_trace(
    go.Scatter(x=scatter_df['index'], y=scatter_df['Africa'], mode='lines+markers', text='Africa', name='Africa')
)
scatter_fig.add_trace(
    go.Scatter(x=scatter_df['index'], y=scatter_df['Europe'], mode='lines+markers', text='Europe', name='Europe')
)
scatter_fig.update_layout(title='Enegry year production per region')
scatter_fig.show()


## Task 2,4

In [149]:
boston_df = pd.read_csv('boston_weather_data.csv', parse_dates=['time'])

,time,tavg,tmin,tmax,prcp,wdir,wspd,pres
0,2013-03-01,3.2,1.1,5.0,0.0,342.0,15.1,1002.7
1,2013-03-02,3.1,1.1,5.6,0.0,307.0,14.4,1004.2
2,2013-03-03,2.6,0.6,6.1,0.0,NaN,14.4,1002.6
3,2013-03-04,1.8,-0.6,5.0,0.0,303.0,28.1,1003.0
4,2013-03-05,2.9,0.0,6.1,0.0,NaN,16.6,1013.5
...,...,...,...,...,...,...,...,...
3648,2023-02-25,-7.9,-10.6,-6.7,0.3,313.0,11.9,1029.5
3649,2023-02-26,-4.2,-7.8,3.9,0.0,268.0,11.9,1012.6
3650,2023-02-27,-1.7,-2.8,-0.6,0.0,46.0,18.7,1014.9
3651,2023-02-28,0.6,-1.7,2.2,7.1,86.0,23.8,1010.7


In [154]:
heatmap = px.imshow(boston_df.corr(numeric_only=True), aspect='auto', text_auto=True)
heatmap.update_layout(title='Bostomn weather correlation heatmap')
heatmap.show()

In [157]:
time_line = px.line(boston_df, x='time', y='tavg', markers=True)
time_line.show()

## Task 5

In [161]:
meteor_df = pd.read_csv('meteorites.csv')
meteor_df.drop(['id', 'name_type', 'class', 'geolocation', 'fall'], inplace=True, axis=1)

In [167]:
meteor_df.dropna(subset=['lat', 'long', 'mass', 'year'], inplace=True)

In [175]:
meteor_sample = meteor_df.sample(100).reset_index().drop(['index'], axis=1)

In [181]:
meteor_sample

,name,mass,year,lat,long
0,Grove Mountains 021598,79.76,2003.0,-72.82500,75.35667
1,Yamato 983270,26.40,1998.0,0.00000,0.00000
2,Grove Mountains 052119,94.91,2006.0,-72.77417,75.31611
3,Meteorite Hills 00596,62.27,2000.0,-79.68333,155.75000
4,Miller Range 090274,49.50,2009.0,0.00000,0.00000
...,...,...,...,...,...
95,Queen Alexandra Range 94422,2.00,1994.0,-84.00000,168.00000
96,Miller Range 090663,31.60,2009.0,0.00000,0.00000
97,Queen Alexandra Range 97716,26.70,1997.0,-84.00000,168.00000
98,Northwest Africa 6673,99.80,2010.0,0.00000,0.00000


In [185]:
map = folium.Map((meteor_sample.loc[50, 'lat'], meteor_sample.loc[50, 'long']), zoom_start=12)
for row in meteor_sample.iterrows():        
    folium.Marker(location=(row[1]['lat'], row[1]['long']),
                  tooltip=row[1]['name'], popup=row[1]['name']).add_to(map)
map